## News

In [1]:
searching_words = ['파이썬', '데이터공방', '코로나', '강의']
for word in searching_words:
    print(word)
    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={word}'
    print(url)

파이썬
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=파이썬
데이터공방
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=데이터공방
코로나
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=코로나
강의
https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=강의


In [8]:
from selenium import webdriver

browser = webdriver.Chrome('./chromedriver')

url = 'https://n.news.naver.com/article/277/0004834732'
browser.get(url)

#url에도 구조가 있다. "주소" 부분과 "파라미터" 부분.
#예를 들어... 네이버에서 무언가를 검색할 때..

#https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=코로나
# ? 기호 앞에가 주소이고, 뒤에가 파라미터다.
# 파라미터에는..
    # ~~?변수=값&변수=값&변수=값.... 이런 식의 진행.
    # 저 위에서 query=코로나. query 변수의 값이 검색어라는 뜻

In [9]:
# url의 html 정보 가져오기
source = browser.page_source #지금 source은 string type

from bs4 import BeautifulSoup as bs #source를 html을 읽을 수 있게 도와주는

soup = bs(source, 'html.parser') #source를 html.parser에 정의된 기준으로 해석하라
soup.select('h2') #soup 데이터 내에서 h3 태그 모두 찾기 => 리스트 형태로 나옴
soup.select('.media_end_head_headline') # 마침표를 앞에 붙이면 "클래스 속성 값"을 찾는다
soup.select('#dic_area') # 샾을 앞에 붙이면 "id 속성 값"을 찾는다
                         # id 속성은 보통 유니크하게 쓰이기 때문에,
                         # 크롤링하고자 하는 데이터에 id 속성이 있으면 '땡큐'하면 된다.

soup.select('h2.media_end_head_headline') #태그 명은 h2이면서,
                                          #클래스 속성은 media_end_head인 것 찾는다.
    
soup.select('div > h2') # div 태그를 부모로 갖고 있는 h2 태그 찾는다.
soup.select('div.media_end_head_title > h2') # div 태그에 media_end_head_title이라는
                                             # 클래스 속성을 부모로 갖는 h2 태그 찾는다.

[<h2 class="media_end_head_headline">[車반도체 수급난]미래車 1대엔 반도체 2000개, 삼성반도체 더 중요해져</h2>]

In [10]:
# 태그에서 값 추출하기
title = soup.select('.media_end_head_headline')[0]
    #soup.select는 항상 list 형태로 나오기 때문에 끝에 [0]을 넣는 것.

title.text #앞뒤에 있는 태그 기호를 모두 없애준다.

'[車반도체 수급난]미래車 1대엔 반도체 2000개, 삼성반도체 더 중요해져'

In [11]:
# 만약 text가 아니라 속성 값을 추출하고 싶다면.. ['속성명']
title['class']

['media_end_head_headline']

## 

In [17]:
import requests
from io import BytesIO
import pandas


user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0"

fs_list = ["연결 재무상태표", "연결 손익계산서", "연결 포괄손익계산서", "연결 자본변동표", "연결 현금흐름표", "재무상태표"]

def download_excel(rcp_no, dcm_no, period, company):
    url = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko".format(rcp_no, dcm_no)
    resp = requests.get(url, headers={"user-agent": user_agent})
    table = BytesIO(resp.content)
    for sheet in fs_list:
        data = pandas.read_excel(table, sheet_name = sheet, skiprows = 5)
        data.to_csv(period + "_" + company + "_" + sheet + ".csv", encoding = "euc-kr")